In [2]:
import tweepy
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from google.cloud import bigquery
import json

client = MongoClient('localhost', 27017)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
bq_client = bigquery.Client()

db = client.worldcup

load_dotenv()

consumer_key = os.getenv('API_KEY')
consumer_secret = os.getenv('API_KEY_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')

In [3]:
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) 

In [7]:
search_query = "#JPN", "#HRV"
no_of_tweets = 100

tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended', count=no_of_tweets).pages(100)

i=0
for tweet in tweets:
    try:
        # content = {
        #     "User": tweet.user.name,
        #     "Location": tweet.user.location,
        #     "Date_Created": tweet.created_at,
        #     "Number_of_Likes": tweet.favorite_count,
        #     "Source_of_Tweet": tweet.source,
        #     "Tweet": tweet.full_text
        # }
        print(tweet)
        # db.jh.insert_one(content)
        i+=1
        if i % 100 == 0:
            print(i)
    except ValueError as e:
        print(e)

AttributeError: 'SearchResults' object has no attribute 'user'

In [ ]:
jpn_vs_hrv = bq_client.get_table('tecky-capstone-project.worldcup.jpn_hrv')

x = db.jh.find()
for row in x:
    row['_id'] = str(row['_id'])
    row['Date_Created'] = str(row['Date_Created'])
    bq_client.insert_rows_json(jpn_vs_hrv,[row])